In [3]:
# JJ Rolling Pivots

import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
# bnf_1hr = pd.read_csv("../data/gold_4hr_tv (2).csv")
# # bnf_1hr = pd.read_csv("../data/gold_1hr_tv.csv")
# # bnf_1hr = pd.read_csv("../data/midcp_select_1hr_tv (4).csv")
# # bnf_1hr = pd.read_csv("../data/fnf_1hr_tv.csv")
# bnf_1hr["datetime"] = pd.to_datetime(bnf_1hr["time"])
# bnf_1hr = bnf_1hr.drop(columns=["time"])
# bnf_1hr = bnf_1hr[(bnf_1hr["datetime"].dt.year >= 2017)]
# bnf_1hr.tail()

In [5]:
spot_data = pd.read_csv('../data/bnf.csv')
spot_data['datetime'] = pd.to_datetime(spot_data['datetime'])
spot_data = pl.DataFrame(spot_data)
spot_data = spot_data.with_columns([pl.col('datetime').alias('index')])

def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label='left',
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )

bnf_resampled = resample(spot_data, '5m', pd.Timedelta(minutes=0))

bnf_df = bnf_resampled.to_pandas()
bnf_1hr = bnf_df

In [34]:
bnf_1hr = bnf_1hr[bnf_1hr['datetime'].dt.year >= 2023]

In [35]:
bnf_1hr.head()

,datetime,open,high,low,close,volume,Bullish Pivot,Bearish Pivot,MA,RSI
111080,2023-01-02 09:15:00,43038.2500,43151.2500,43030.0500,43030.0500,0,NaN,NaN,43097.7840,62.7796
111081,2023-01-02 09:20:00,43044.6000,43111.3500,42987.4500,43086.2000,0,NaN,1.0000,43101.2395,82.7247
111082,2023-01-02 09:25:00,43078.2500,43086.2500,42973.6500,42997.9500,0,NaN,NaN,43103.5465,30.3760
111083,2023-01-02 09:30:00,43000.8500,43162.0000,42965.2000,43152.0000,0,NaN,NaN,43106.7935,68.4914
111084,2023-01-02 09:35:00,43153.2500,43248.6500,43123.7500,43248.6500,0,1.0000,NaN,43110.7980,77.6639


In [36]:
PORTFOLIO = 50_00_000
INDEX_LEVERAGE = 6
SLIPPAGE_FACTOR = 0.0001

In [37]:
def generate_pivots2(df):

    # df["Bullish Pivot"] = 0
    # df["Bearish Pivot"] = 0
    # df["MA"] = df["close"].rolling(window=18).mean()

    # Generate signals using boolean masking
    bull_pivot_mask = (df["low"] >= df["low"].shift(1)) & (
        df["low"].shift(2) >= df["low"].shift(1)
    )
    bear_pivot_mask = (df["high"] <= df["high"].shift(1)) & (
        df["high"].shift(2) <= df["high"].shift(1)
    )

    df.loc[bull_pivot_mask, "Bullish Pivot"] = 1
    df.loc[bear_pivot_mask, "Bearish Pivot"] = 1

    return df


bnf_1hr = generate_pivots2(bnf_1hr)
bnf_1hr.head()

,datetime,open,high,low,close,volume,Bullish Pivot,Bearish Pivot,MA,RSI
111080,2023-01-02 09:15:00,43038.2500,43151.2500,43030.0500,43030.0500,0,NaN,NaN,43097.7840,62.7796
111081,2023-01-02 09:20:00,43044.6000,43111.3500,42987.4500,43086.2000,0,NaN,1.0000,43101.2395,82.7247
111082,2023-01-02 09:25:00,43078.2500,43086.2500,42973.6500,42997.9500,0,NaN,NaN,43103.5465,30.3760
111083,2023-01-02 09:30:00,43000.8500,43162.0000,42965.2000,43152.0000,0,NaN,NaN,43106.7935,68.4914
111084,2023-01-02 09:35:00,43153.2500,43248.6500,43123.7500,43248.6500,0,1.0000,NaN,43110.7980,77.6639


In [86]:
def add_ma(df, ma_period=100):
    df['MA'] = df['close'].rolling(window=ma_period).mean()
    return df

bnf_1hr = add_ma(bnf_1hr)

In [114]:
def calculate_rsi(df, column="close", period=40):

    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


bnf_1hr["RSI"] = calculate_rsi(bnf_1hr)

In [115]:
percentiles = [0.1, 0.2, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

percentile_values = bnf_1hr['RSI'].quantile(percentiles)

print("Percentile Distribution of 'High Entry Ratio':")
print(percentile_values)

Percentile Distribution of 'High Entry Ratio':
0.1000   36.4231
0.2000   41.5178
0.2500   43.3434
0.5000   50.8686
0.7500   58.1003
0.9000   64.9548
0.9500   68.9801
0.9900   75.3393
Name: RSI, dtype: float64


In [116]:
def execute_trades_long(df):
    tradebook = []

    in_trade = False
    long_trade_active = False
    short_trade_active = False

    trailing_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    long_sl_pct = 0.4
    long_target_pct = 100

    rsi_upper_range = 90
    rsi_lower_range = 28

    long_points = 0
    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i - 1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        # print(current_candle_time, df.iloc[i]['RSI'])

        if not in_trade:
            if df.iloc[i - 1]["Bullish Pivot"] == 1:
                # Long Side
                signal_time = previous_candle_time

                if current_candle_high >= previous_candle_high and (
                    df.iloc[i - 1]["RSI"] <= rsi_lower_range
                ):
                    if (current_candle_open > previous_candle_high) and (current_candle_time.date() > previous_candle_time.date()):
                        # Check Entry Skip on Next Day Gap Up Open
                        # print("Long Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        # print("Long Entry Triggered", current_candle_time)
                        in_trade = True
                        long_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_high
                        # initial_sl = df.iloc[i - 2]["low"] * 0.9975
                        initial_sl = entry_price * (1 - long_sl_pct / 100)
                        long_target = entry_price * (1 + long_target_pct / 100)
                        qty = 0.01 * PORTFOLIO / abs(entry_price - initial_sl)

        if in_trade:
            if long_trade_active:
                
                if (current_candle_low > df.iloc[i]['MA']) and (previous_candle_low > df.iloc[i-1]['MA']):
                    trailing_active = True

                if trailing_active:

                    trailing_ma = df.iloc[i]['MA']
                    
                    if (current_candle_open < initial_sl) and (current_candle_time.date() > entry_time.date()):
                        # Gap Outside Initial SL
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        long_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)

                    elif current_candle_close <= trailing_ma:
                        #TSL Hit
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Trailing SL Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= long_target:
                        in_trade = False
                        long_trade_active = False
                        exit_price = long_target
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    # elif (df.iloc[i-1]["RSI"] >= rsi_upper_range) and (df.iloc[i]["RSI"] <= rsi_upper_range):
                    #     in_trade = False
                    #     long_trade_active = False
                    #     exit_price = current_candle_close
                    #     exit_time = current_candle_time
                    #     long_points = exit_price - entry_price
                    #     remark = "RSI Overbought"

                else:
                    
                    if (current_candle_open < initial_sl) and (current_candle_time.date() > entry_time.date()):
                        # Gap Outside Initial SL
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        long_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= long_target:
                        in_trade = False
                        long_trade_active = False
                        exit_price = long_target
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] >= rsi_upper_range) and (df.iloc[i]["RSI"] <= rsi_upper_range):
                        in_trade = False
                        long_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        long_points = exit_price - entry_price
                        remark = "RSI Overbought"
                        
                if long_points:
                    trade = {
                        "Trade Type": "LONG",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": long_points,
                        "Points w CS": long_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * long_points,
                        "PnL w CS": qty
                        * (
                            long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * long_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (long_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    long_points = 0
                    long_trade_active = False
                    trailing_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df

In [117]:
tb_long = execute_trades_long(bnf_1hr)

In [118]:
tb_long['ROI% w CS'].sum()

-18.994281728242086

In [119]:
tb_targets = tb_long[tb_long['Remarks'] == 'Target Hit']
tb_rsi_overbought = tb_long[tb_long['Remarks'] == 'RSI Overbought']
tb_sl = tb_long[tb_long['Remarks'] == 'Initial SL Hit']
tb_gap = tb_long[tb_long['Remarks'] == 'Gap Outside Initial SL']
tb_tsl = tb_long[tb_long['Remarks'] == 'Trailing SL Hit']
len(tb_targets) , len(tb_rsi_overbought) , len(tb_sl) , len(tb_gap) , len(tb_tsl)

(0, 0, 23, 2, 8)

In [120]:
stats_df5 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)

new_tb = tb_long

# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = new_tb[(new_tb["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df5.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df5["Total ROI"].sum()
overall_total_trades = stats_df5["Total Trades"].sum()
overall_win_rate = (new_tb["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = new_tb[new_tb["ROI% w CS"] > 0]["ROI% w CS"].mean()
overall_avg_loss = new_tb[new_tb["ROI% w CS"] < 0]["ROI% w CS"].mean()
overall_max_drawdown = (
    new_tb["ROI% w CS"].cumsum() - new_tb["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df5.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df5

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,0.0000,0,NaN,NaN,NaN,NaN,NaN
2018,0.0000,0,NaN,NaN,NaN,NaN,NaN
2019,0.0000,0,NaN,NaN,NaN,NaN,NaN
2020,0.0000,0,NaN,NaN,NaN,NaN,NaN
2021,0.0000,0,NaN,NaN,NaN,NaN,NaN
2022,0.0000,0,NaN,NaN,NaN,NaN,NaN
2023,-12.2905,22,22.7273,1.3224,-1.1119,-14.7480,-0.8334
2024,-6.7037,11,18.1818,1.1510,-1.0006,-5.6538,-1.1857
Overall,-18.9943,33.0000,21.2121,1.2734,-1.0734,-17.9444,-1.0585


In [75]:
tb_long.tail()

,Trade Type,Signal Generated At,Entry Time,Entry Price,Initial SL,Exit Time,Exit Price,Remarks,Points Captured,Points w CS,Qty,Leverage,PnL,PnL w CS,ROI%,ROI% w CS,Trade Year
322,LONG,2024-05-30 11:45:00,2024-05-30 11:50:00,48598.3500,48403.9566,2024-05-30 14:30:00,48744.0000,Trailing SL Hit,145.6500,135.9158,257.2104,6,37462.6916,34958.9454,0.7493,0.6992,2024
323,LONG,2024-05-30 14:40:00,2024-05-30 14:45:00,48758.2500,48563.2170,2024-05-30 15:00:00,48563.2170,Initial SL Hit,-195.0330,-204.7651,256.3669,6,-50000.0000,-52495.0000,-1.0000,-1.0499,2024
324,LONG,2024-05-31 14:35:00,2024-05-31 14:40:00,48867.5000,48672.0300,2024-06-03 13:35:00,50731.4000,Trailing SL Hit,1863.9000,1853.9401,255.7937,6,476773.9295,474226.2521,9.5355,9.4845,2024
325,LONG,2024-06-04 09:50:00,2024-06-04 09:55:00,49402.1000,49204.4916,2024-06-04 10:00:00,49204.4916,Initial SL Hit,-197.6084,-207.4691,253.0257,6,-50000.0000,-52495.0000,-1.0000,-1.0499,2024
326,LONG,2024-06-04 12:30:00,2024-06-04 12:35:00,46625.6500,46439.1474,2024-06-04 12:35:00,46439.1474,Initial SL Hit,-186.5026,-195.8091,268.0928,6,-50000.0000,-52495.0000,-1.0000,-1.0499,2024


In [76]:
# tb_long.to_csv('Parle-G Long 5_89_20 1pt5 sl.csv')

In [77]:
def calculate_rsi_short(df, column="close", period=7):

    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi


bnf_1hr["RSI"] = calculate_rsi_short(bnf_1hr)

In [78]:
percentiles = [0.1, 0.2, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

percentile_values = bnf_1hr['RSI'].quantile(percentiles)

print("Percentile Distribution of 'High Entry Ratio':")
print(percentile_values)

Percentile Distribution of 'High Entry Ratio':
0.1000   20.2989
0.2000   29.9210
0.2500   34.0788
0.5000   50.8781
0.7500   67.2599
0.9000   81.0455
0.9500   87.7047
0.9900   96.7890
Name: RSI, dtype: float64


In [183]:
def execute_trades_short(df):
    tradebook = []

    in_trade = False
    short_trade_active = False

    trailing_active = False

    remark = ""

    signal_entry_price = 1_00_000
    signal_initial_sl = 0

    points = 0

    short_sl_pct = 4
    short_target_pct = 100

    rsi_upper_range = 80
    rsi_lower_range = 25

    short_points = 0

    for i in range(0, len(df)):

        points = 0

        previous_candle_open = df.iloc[i - 1]["open"]
        previous_candle_high = df.iloc[i - 1]["high"]
        previous_candle_low = df.iloc[i - 1]["low"]
        previous_candle_close = df.iloc[i - 1]["close"]
        previous_candle_time = df.iloc[i - 1]["datetime"]
        previous_candle = df.iloc[i - 1]

        current_candle_open = df.iloc[i]["open"]
        current_candle_high = df.iloc[i]["high"]
        current_candle_low = df.iloc[i]["low"]
        current_candle_close = df.iloc[i]["close"]
        current_candle_time = df.iloc[i]["datetime"]
        current_candle = df.iloc[i]

        print(current_candle_time, df.iloc[i]['RSI'])

        if not in_trade:
            if df.iloc[i - 1]["Bearish Pivot"] == 1:
                # Short Side
                signal_time = previous_candle_time

                if current_candle_low <= previous_candle_low and (
                    df.iloc[i - 1]["RSI"] >= rsi_upper_range
                ):
                    if (current_candle_open < previous_candle_low) and (current_candle_time.date() > previous_candle_time.date()):
                        # Check Entry Skip on Next Day Gap Up Open
                        # print("Short Entry Skipped", current_candle_time)
                        in_trade = False
                        signal_entry_price = 1_00_000
                        signal_initial_sl = 0
                        continue
                    else:
                        # Entry Triggered
                        # print("Short Entry Triggered", current_candle_time)
                        in_trade = True
                        short_trade_active = True
                        entry_time = current_candle_time
                        entry_price = previous_candle_low
                        # initial_sl = min(
                        #     df.iloc[i - 2]["low"], entry_price * (1 - short_sl_pct / 100)
                        # )
                        initial_sl = entry_price * (1 + short_sl_pct / 100)
                        short_target = entry_price * (1 - short_target_pct / 100)
                        qty = PORTFOLIO * INDEX_LEVERAGE / entry_price

        if in_trade:
            if short_trade_active:
                
                if current_candle_high < df.iloc[i]['MA']:
                    trailing_active = True

                if trailing_active:

                    trailing_ma = df.iloc[i]['MA']
                    
                    if current_candle_open > initial_sl:
                        # Gap Outside Initial SL
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        short_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)

                    # elif current_candle_close <= trailing_ma:
                    #     #TSL Hit
                    #     in_trade = False
                    #     short_trade_active = False
                    #     exit_price = current_candle_close
                    #     exit_time = current_candle_time
                    #     short_points = exit_price - entry_price
                    #     remark = "Trailing SL Hit"
                    #     # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= short_target:
                        in_trade = False
                        short_trade_active = False
                        exit_price = short_target
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] <= rsi_lower_range) and (df.iloc[i]["RSI"] >= rsi_lower_range):
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "RSI Oversold"

                else:
                    
                    if current_candle_open > initial_sl:
                        # Gap Outside Initial SL
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_open
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Gap Outside Initial SL"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_high >= initial_sl:
                        # Initial SL Hit
                        in_trade = False
                        short_trade_active = False
                        exit_price = initial_sl
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Initial SL Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif current_candle_low <= short_target:
                        in_trade = False
                        short_trade_active = False
                        exit_price = short_target
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "Target Hit"
                        # print(remark, current_candle_time, exit_price)
    
                    elif (df.iloc[i-1]["RSI"] <= rsi_lower_range) and (df.iloc[i]["RSI"] >= rsi_lower_range):
                        in_trade = False
                        short_trade_active = False
                        exit_price = current_candle_close
                        exit_time = current_candle_time
                        short_points = entry_price - exit_price
                        remark = "RSI Oversold"
                        
                if short_points:
                    trade = {
                        "Trade Type": "SHORT",
                        "Signal Generated At": signal_time,
                        "Entry Time": entry_time,
                        "Entry Price": entry_price,
                        "Initial SL": initial_sl,
                        "Exit Time": exit_time,
                        "Exit Price": exit_price,
                        "Remarks": remark,
                        "Points Captured": short_points,
                        "Points w CS": short_points
                        - (SLIPPAGE_FACTOR * (entry_price + exit_price)),
                        "Qty": qty,
                        "Leverage": INDEX_LEVERAGE,
                        "PnL": qty * short_points,
                        "PnL w CS": qty
                        * (
                            short_points - (SLIPPAGE_FACTOR * (entry_price + exit_price))
                        ),
                        "ROI%": qty * short_points * 100 / PORTFOLIO,
                        "ROI% w CS": qty
                        * (short_points - (SLIPPAGE_FACTOR * (entry_price + exit_price)))
                        * 100
                        / PORTFOLIO,
                        "Trade Year": entry_time.year,
                    }
                    tradebook.append(trade)

                    short_points = 0
                    short_trade_active = False

    trade_book_df = pd.DataFrame(tradebook)
    return trade_book_df

In [1258]:
tb_short = execute_trades_short(bnf_1hr)

2017-01-02 09:00:00+05:30 nan
2017-01-02 13:00:00+05:30 0.0
2017-01-02 17:00:00+05:30 0.0
2017-01-02 21:00:00+05:30 90.47619047619048
2017-01-03 09:00:00+05:30 65.51724137931035
2017-01-03 13:00:00+05:30 91.37931034482759
2017-01-03 17:00:00+05:30 83.46456692913387
2017-01-03 21:00:00+05:30 35.690235690235696
2017-01-04 09:00:00+05:30 35.09933774834437
2017-01-04 13:00:00+05:30 35.64356435643565
2017-01-04 17:00:00+05:30 28.896103896103895
2017-01-04 21:00:00+05:30 41.06145251396648
2017-01-05 09:00:00+05:30 22.140221402214024
2017-01-05 13:00:00+05:30 30.069930069930066
2017-01-05 17:00:00+05:30 77.44360902255639
2017-01-05 21:00:00+05:30 69.5945945945946
2017-01-06 09:00:00+05:30 70.96774193548387
2017-01-06 13:00:00+05:30 87.5
2017-01-06 17:00:00+05:30 64.02877697841726
2017-01-06 21:00:00+05:30 69.1358024691358
2017-01-09 09:00:00+05:30 58.10810810810811
2017-01-09 13:00:00+05:30 40.828402366863905
2017-01-09 17:00:00+05:30 36.31578947368421
2017-01-09 21:00:00+05:30 30.76923076923

In [1259]:
tb_targets_short = tb_short[tb_short['Remarks'] == 'Target Hit']
tb_rsi_oversold_short = tb_short[tb_short['Remarks'] == 'RSI Oversold']
tb_sl_short = tb_short[tb_short['Remarks'] == 'Initial SL Hit']
len(tb_targets_short) , len(tb_rsi_oversold_short) , len(tb_sl_short)

(0, 72, 24)

In [1260]:
tb_short['ROI% w CS'].sum()

271.95093269507146

In [1261]:
# tb_short.tail(25)

In [1262]:
stats_df5 = pd.DataFrame(
    index=range(2017, 2025),
    columns=[
        "Total ROI",
        "Total Trades",
        "Win Rate",
        "Avg Profit% per Trade",
        "Avg Loss% per Trade",
        "Max Drawdown",
        "ROI/DD Ratio",
    ],
)

new_tb = tb_short

# Iterate over each year
for year in range(2017, 2025):
    # Filter trades for the current year
    year_trades = new_tb[(new_tb["Trade Year"] == year)]

    # Calculate total ROI
    total_roi = year_trades["ROI% w CS"].sum()

    # Calculate total number of trades
    total_trades = len(year_trades)

    # Calculate win rate
    win_rate = (year_trades["ROI% w CS"] > 0).mean() * 100

    # Calculate average profit per trade
    avg_profit = year_trades[year_trades["ROI% w CS"] > 0]["ROI% w CS"].mean()

    # Calculate average loss per trade
    avg_loss = year_trades[year_trades["ROI% w CS"] < 0]["ROI% w CS"].mean()

    # Calculate maximum drawdown
    max_drawdown = (
        year_trades["ROI% w CS"].cumsum() - year_trades["ROI% w CS"].cumsum().cummax()
    ).min()

    # Calculate ROI/DD ratio
    roi_dd_ratio = total_roi / abs(max_drawdown)

    # Store the statistics in the DataFrame
    stats_df5.loc[year] = [
        total_roi,
        total_trades,
        win_rate,
        avg_profit,
        avg_loss,
        max_drawdown,
        roi_dd_ratio,
    ]

# Calculate overall statistics
overall_total_roi = stats_df5["Total ROI"].sum()
overall_total_trades = stats_df5["Total Trades"].sum()
overall_win_rate = (new_tb["ROI% w CS"] > 0).mean() * 100
overall_avg_profit = new_tb[new_tb["ROI% w CS"] > 0]["ROI% w CS"].mean()
overall_avg_loss = new_tb[new_tb["ROI% w CS"] < 0]["ROI% w CS"].mean()
overall_max_drawdown = (
    new_tb["ROI% w CS"].cumsum() - new_tb["ROI% w CS"].cumsum().cummax()
).min()
overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)

# Store the overall statistics in the DataFrame
stats_df5.loc["Overall"] = [
    overall_total_roi,
    overall_total_trades,
    overall_win_rate,
    overall_avg_profit,
    overall_avg_loss,
    overall_max_drawdown,
    overall_roi_dd_ratio,
]
stats_df5

/tmp/ipykernel_164176/2507184954.py:42: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio
2017,13.8554,14,57.1429,7.2130,-7.3082,-13.3272,1.0396
2018,-21.2757,16,50.0000,6.1813,-8.8408,-31.9097,-0.6667
2019,47.1204,15,60.0000,10.8025,-8.3504,-13.5479,3.4781
2020,213.1726,10,90.0000,23.8242,-1.2451,0.0000,inf
2021,19.7711,16,68.7500,5.3097,-7.7271,-26.4962,0.7462
2022,-1.7660,9,55.5556,9.3447,-12.1224,-15.1390,-0.1167
2023,-27.9187,12,50.0000,6.0232,-10.6763,-36.3672,-0.7677
2024,28.9918,4,75.0000,10.1509,-1.4610,-1.4610,19.8445
Overall,271.9509,96.0000,61.4583,10.0088,-8.6099,-53.7252,5.0619
